In [11]:
import random

def printSolution(solution, log_file):
    log_to_file(log_file, f"Final Solution: {solution}")
    board_str = "\n"
    
    for i in range(8):
        x = solution[i] - 1
        for j in range(8):
            if j == x:
                board_str += '[Q]'
            else:
                board_str += '[ ]'
        board_str += '\n'
    
    log_to_file(log_file, board_str)

def getHeuristic(instance):
    heuristic = []
    for i in range(len(instance)):
        j = i - 1
        heuristic.append(0)
        while j >= 0:
            if instance[i] == instance[j] or (abs(instance[i] - instance[j]) == abs(i - j)):
                heuristic[i] += 1
            j -= 1
        j = i + 1
        while j < len(instance):
            if instance[i] == instance[j] or (abs(instance[i] - instance[j]) == abs(i - j)):
                heuristic[i] += 1
            j += 1
    return heuristic

def getFitness(instance):
    clashes = 0
    for i in range(len(instance) - 1):
        for j in range(i + 1, len(instance)):
            if instance[i] == instance[j]:
                clashes += 1
    for i in range(len(instance) - 1):
        for j in range(i + 1, len(instance)):
            if abs(instance[j] - instance[i]) == abs(j - i):
                clashes += 1
    return 28 - clashes

def crossover(parent1, parent2, crossOverPoint):
    offspring = []
    for i in range(crossOverPoint):
        offspring.append(parent1[i])
    for i in range(crossOverPoint, 8):
        offspring.append(parent2[i])
    return offspring

def crossOverParents(crossOverPoint):
    global parent1, parent2, child1, child2
    child1 = crossover(parent1, parent2, crossOverPoint)
    child2 = crossover(parent2, parent1, crossOverPoint)

def mutate(instance):
    newChange = -1
    while newChange != 0:
        newChange = 0
        tmpInstance = instance[:]
        heuristics = getHeuristic(tmpInstance)
        index = heuristics.index(max(heuristics))
        maxFitness = getFitness(tmpInstance)
        for i in range(1, 9):
            tmpInstance[index] = i
            if getFitness(tmpInstance) > maxFitness:
                maxFitness = getFitness(tmpInstance)
                newChange = i
            tmpInstance = instance[:]
        if newChange == 0:
            for i in range(len(instance) - 1):
                for j in range(i + 1, len(instance)):
                    if instance[i] == instance[j]:
                        instance[j] = random.randint(1, 8)
        else:
            instance[index] = newChange

def log_to_file(filename, message):
    with open(filename, 'a') as f:  # 'a' mode appends to the file
        f.write(message + '\n')

if __name__ == "__main__":
    log_file = "genetic_algorithm_log.txt"
    log_to_file(log_file, "*** 8 Queens Problem using Genetic Algorithm ***")
    
    crossOverPoint = 4
    log_to_file(log_file, f"Initial Crossover Point: {crossOverPoint}")
    #population = [random_chromosome(8) for _ in range(50)]

    parent1 = [random.randint(1, 8) for _ in range(8)]
    parent2 = [random.randint(1, 8) for _ in range(8)]
    #parent1 = tournament_selection(population)
    #log_to_file(log_file, f"Parent 1: {parent1}")
    #parent2 = tournament_selection(population)
    log_to_file(log_file, f"Initial Parent 1: {parent1}")
    log_to_file(log_file, f"Initial Parent 2: {parent2}")
    
    fitnessParent1 = getFitness(parent1)
    fitnessParent2 = getFitness(parent2)
    
    while fitnessParent1 != 28 and fitnessParent2 != 28:
        crossOverParents(crossOverPoint)
        log_to_file(log_file, f"CrossOvered Child 1: {child1}")
        log_to_file(log_file, f"CrossOvered Child 2: {child2}")
        
        mutate(child1)
        mutate(child2)
        
        fitnessParent1 = getFitness(child1)
        fitnessParent2 = getFitness(child2)
        
        log_to_file(log_file, f"Fitness of new Parent 1: {fitnessParent1}")
        log_to_file(log_file, f"Fitness new Parent  2: {fitnessParent2}")
        
        if fitnessParent1 >= fitnessParent2:
            parent1 = child1
        else:
            parent2 = child2

        log_to_file(log_file, f"Selected Parent 1: {parent1}")
        log_to_file(log_file, f"Selected Parent 2: {parent2}")
    
    solution = parent1 if getFitness(parent1) == 28 else parent2
    printSolution(solution, log_file)
